## **Identification de langue avec le Deep Learning et**
## **un embedding en input**

In [1]:
import numpy as np
import pandas as pd
import random
import joblib
import pickle
import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Choix de la Tokenisation (1 = Keras, 2 = BERT, 3 = Tiktoken)
sel_tokenization = 3

## Pour résoudre les problème de mémoire et de performances
max_length = 500
# nb_phrase_lang = 100000

import warnings
warnings.filterwarnings('ignore')

#### **Lectures des phrases et de leur étiquette "Langue" pour les langues sélectionnées**

In [2]:
# Ouvrir le fichier d'entrée en mode lecture
def create_lang_df(path):
    df = pd.read_csv(path, index_col ='id')
    return df

df_big = create_lang_df('../data/multilingue/sentences.csv')
lan_code = ['eng','fra','deu','spa','ita']
df = pd.DataFrame(columns=df_big.columns)
for i in range(len(lan_code)):
    df= pd.concat([df, df_big[df_big['lan_code']==lan_code[i]]]) #.iloc[:nb_phrase_lang]])
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
n_rows = len(df)
print('Nombre de lignes de sentence.csv:',n_rows)

df

Nombre de lignes de sentence.csv: 1750000


,lan_code,sentence
0,spa,El poeta intentó suicidarse en su estudio.
1,fra,Je me suis sentie un peu prise de vertige.
2,deu,Ich reise im kommenden Jahr nach Rio.
3,ita,Spero che verrete alla mia festa di compleanno.
4,ita,La telefonata era un trucco per farlo uscire d...
...,...,...
1749995,eng,What a strange message! There is no sender and...
1749996,ita,Noi ci stiamo per fare una doccia.
1749997,eng,Two girls were hanging on to Tom's arms.
1749998,fra,Tu dois cesser de t'énerver après des choses q...


#### **Réalisation d'un jeu de données d'entrainement et de test**

In [3]:
# créer 2 dataframes: 1 train (70% des phrases) et 1 test (30% des phrases)
n_train = int(n_rows*0.7)
df_train = df.iloc[:n_train].sample(frac=1, random_state=42).reset_index(drop=True)
df_test = df.iloc[n_train:].sample(frac=1, random_state=24).reset_index(drop=True)
pd.set_option('display.max_colwidth', 150)
# display(df_train)

nb_phrases_lang =[]
for l in lan_code: # range(len(lan_code)):
    nb_phrases_lang.append(sum(df_train['lan_code']==l))
print("Nombre de phrases par langue dans l'ensemble Train :",nb_phrases_lang)

Nombre de phrases par langue dans l'ensemble Train : [244836, 245461, 244737, 244903, 245063]


#### **Selection du Tokenizer,**
#### **Encodage et padding du text avec le tokenizer**

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Selection du tokenizer
if sel_tokenization==3:
    import tiktoken
    tokenizer = tiktoken.get_encoding("cl100k_base")
elif sel_tokenization==2:
    from transformers import BertTokenizerFast
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')
else:
    from tensorflow.keras.preprocessing.text import Tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df['sentence'])

# Données d'exemple (textes et leurs langues correspondantes)
textes = df_train['sentence']    
langues = df_train['lan_code']
    
# Encodage des étiquettes (langues)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(langues)
    
def encode_text(textes):
    global max_length, nb_unique_tokens
    
    if sel_tokenization==3:
        sequences = tokenizer.encode_batch(textes)
        nb_unique_tokens = tokenizer.max_token_value + 1
    elif sel_tokenization==2:
        textes = textes.tolist()
        sequences = tokenizer.batch_encode_plus(textes).input_ids
        nb_unique_tokens = len(set(tokenizer.get_vocab()))
    else:
        sequences = tokenizer.texts_to_sequences(textes)
        nb_unique_tokens = len(tokenizer.word_index)
    return pad_sequences(sequences, maxlen=max_length, padding='post')

#### **Definition du modèle d'identification et encodage de l'ensemble Train**

In [5]:
# Padding des séquences
padded_sequences = encode_text(textes) # pad_sequences(sequences, maxlen=max_length, padding='post')
print("Nombre de tokens uniques :",nb_unique_tokens)
print("======")
    
# Conversion des étiquettes en catégories one-hot
labels_one_hot = to_categorical(labels_encoded)

# Création du modèle LSTM
model = Sequential()
model.add(Embedding(input_dim=nb_unique_tokens, output_dim=200, input_length=max_length))
model.add(GlobalAveragePooling1D())  
model.add(Dense(units = 400, activation = "tanh", kernel_initializer='glorot_uniform', name = "Dense_1"))
model.add(Dense(units = 200, activation = "tanh", kernel_initializer='glorot_uniform', name = "Dense_2"))
model.add(Dense(units = 100, activation = "tanh", kernel_initializer='glorot_uniform', name = "Dense_3"))
model.add(Dense(units = 50, activation = "tanh", kernel_initializer='glorot_uniform', name = "Dense_4"))

# model.add(LSTM(100))
model.add(Dense(5, activation='softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Nombre de tokens uniques : 100277
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 200)          20055400  
                                                                 
 global_average_pooling1d (G  (None, 200)              0         
 lobalAveragePooling1D)                                          
                                                                 
 Dense_1 (Dense)             (None, 400)               80400     
                                                                 
 Dense_2 (Dense)             (None, 200)               80200     
                                                                 
 Dense_3 (Dense)             (None, 100)               20100     
                                                                 
 Dense_4 (Dense)             (None, 50)                5050      
                      

#### **Entraînement du modèle**

In [ ]:
#### Entraînement du modèle
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
model.fit(padded_sequences, labels_one_hot, epochs=20, validation_split=0.1, batch_size=128, verbose=1, callbacks=[stop_early])

#### **Sauvegarde et/ou Chargement du modele**

In [7]:
# definition du nom de fichier de sauvegarde
def get_file_name(sel_tokenization):
    if sel_tokenization == 3: return("../data/dl_tiktoken_id_language_model.h5")
    elif sel_tokenization == 2: return("../data/dl_BERT_id_language_model.h5")
    else: return("../data/dl_default_id_language_model.h5")


# Sauvegarde du modèle entrainé
# model.save(get_file_name(sel_tokenization))
# if sel_tokenization==1:
#     with open('../data/tokenizer_Keras.pkl', 'wb') as tokenizer_file:
#         pickle.dump(tokenizer, tokenizer_file)

# Chargement d'un modèle pré-entrainé
# model = keras.models.load_model(get_file_name(sel_tokenization))
# if sel_tokenization==1:
#     with open('../data/tokenizer_Keras.pkl', 'rb') as tokenizer_file:
#         tokenizer = pickle.load(tokenizer_file)


#### **Test de l'efficacité du modèle**

In [8]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
import random

# Préparation des nouvelles données à prédire
textes_test = df_test['sentence']
langues_test = df_test['lan_code']

# Prédiction des langues des nouveaux textes
predictions = model.predict(encode_text(textes_test))

# Décodage des prédictions en langues
predicted_labels_encoded = np.argmax(predictions, axis=1)
predicted_languages = label_encoder.classes_[predicted_labels_encoded]
print("======")

print(classification_report(langues_test,predicted_languages))
print("======")

display(pd.crosstab(langues_test,predicted_languages,rownames=['Classe réelle'], colnames=['Classe prédite']))
accuracy_clf = accuracy_score(langues_test,predicted_languages)
print("Accuracy Classifier = {:.3f}".format(accuracy_clf))
print("======")


16407/16407 [==============================] - 14s 863us/step
              precision    recall  f1-score   support

         deu       1.00      1.00      1.00    105263
         eng       1.00      1.00      1.00    105164
         fra       1.00      1.00      1.00    104539
         ita       1.00      1.00      1.00    104937
         spa       1.00      1.00      1.00    105097

    accuracy                           1.00    525000
   macro avg       1.00      1.00      1.00    525000
weighted avg       1.00      1.00      1.00    525000



Classe prédite,deu,eng,fra,ita,spa
Classe réelle,,,,,
deu,105158,69,10,17,9
eng,6,105129,7,12,10
fra,4,102,104356,26,51
ita,5,67,23,104720,122
spa,6,83,21,177,104810


Accuracy Classifier = 0.998


#### **Affichage d'exemples**

In [9]:
# Affichage des prédictions
print("Exemples de prédiction de langue:")
print("Réelle\t- Prédite - Texte")
n_test = min(len(textes_test),10)
for _ in range(n_test):
    i = random.randint(0, len(textes_test))
    print(f" {langues_test[i]}\t- {predicted_languages[i]}     -'{textes_test[i].ljust(120, '.')[:120]}'")

Exemples de prédiction de langue:
Réelle	- Prédite - Texte
 spa	- spa     -'Tom abrió la canilla....................................................................................................'
 fra	- fra     -'C'est une nouveauté.....................................................................................................'
 fra	- fra     -'Je me souviens de lorsque j'avais à peu près ton âge....................................................................'
 deu	- deu     -'Sie haben in Ihrem Zimmer Kerzen angezündet.............................................................................'
 fra	- fra     -'Vous avez eu plusieurs occasions de parler anglais......................................................................'
 eng	- eng     -'Tom doesn't want to be seen talking to you..............................................................................'
 deu	- deu     -'Wir durchkämmten die Polizeiaufzeichnungen des Vorfalls, fanden aber keine Erwähnung von Z

In [10]:
# Affichage de mauvaises prédictions
print("Exemples de mauvaises prédictions de langue:")
list_bad = []
n = len(textes_test)
if n>0:
    for i in range(n):
        if predicted_languages[i] != langues_test[i] :
            list_bad.append(i)
    print("Réelle\t- Prédite - Texte")
    n_test = min(n,10)
    for _ in range(n_test):
        i = random.randint(0, len(list_bad))
        print(f" {langues_test[list_bad[i]]}\t- {predicted_languages[list_bad[i]]}     -'{textes_test[list_bad[i]].ljust(120, '.')[:120]}'")
else:
    print("Félicitations !!!! Le modèle n'a fait aucune mauvaise prédictions.")

Exemples de mauvaises prédictions de langue:
Réelle	- Prédite - Texte
 spa	- ita     -'Le di carne a mi perro..................................................................................................'
 fra	- ita     -'Tire la chevillette, la bobinette cherra................................................................................'
 deu	- ita     -'Oder?...................................................................................................................'
 ita	- spa     -'Poco importa............................................................................................................'
 spa	- ita     -'Mary calza un 37........................................................................................................'
 fra	- eng     -'Exactement..............................................................................................................'
 spa	- ita     -'Tu madre come mierda...........................................................